In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd

# Ruta de la carpeta de imágenes y del CSV en Google Drive
image_dir = '/content/drive/My Drive/Proyecto-DataSets/imagenes_claras'
csv_file = '/content/drive/My Drive/Proyecto-DataSets/dataset.csv'

# Cargar el CSV
df = pd.read_csv(csv_file)
df['image'] = df['image'].astype(str).fillna('')

# Listas para almacenar imágenes válidas y filas válidas
valid_images = []
valid_rows = []

# Verificar la existencia de cada imagen en el CSV
for i, row in df.iterrows():
    image_path = os.path.join(image_dir, row['image'])
    if os.path.exists(image_path):
        valid_images.append(row['image'])
        valid_rows.append(row)

# Crear un nuevo DataFrame con solo las filas válidas
df_clean = pd.DataFrame(valid_rows)

# Guardar el nuevo CSV limpio en Google Drive
clean_csv_path = '/content/drive/My Drive/Proyecto-DataSets/dataset_clean.csv'
df_clean.to_csv(clean_csv_path, index=False)

print(f"Total de imágenes válidas: {len(valid_images)}")
print(f"Nuevo CSV generado en: {clean_csv_path}")

Total de imágenes válidas: 2596
Nuevo CSV generado en: /content/drive/My Drive/Proyecto-DataSets/dataset_clean.csv


In [3]:
import tensorflow as tf
import pandas as pd # pandas es para manejar el .csv labels y eso, de manera eficiente.
import os
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = '/content/drive/My Drive/Proyecto-DataSets/imagenes_claras'
csv_file = '/content/drive/My Drive/Proyecto-DataSets/dataset_clean.csv'
df = pd.read_csv(csv_file)

datagen = ImageDataGenerator(
    rescale=1./255, # normalizacion los valores de píxel entre 0 y 1
    validation_split=0.2 # % 80 training, 20 % test
)

df['image'] = df['image'].astype(str)
df['label'] = df['label'].astype(str)

# training
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=dataset_path,
    x_col="image",
    y_col="label",
    target_size=(224, 224),  # estandar
    class_mode="categorical",  # clasificación multiclase
    batch_size=32,  # tamaño de batch
    subset="training",  # usar esta parte para el entrenamiento
    shuffle=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# validacion/test
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=dataset_path,
    x_col="image",
    y_col="label",
    target_size=(224, 224), # estandar
    class_mode="categorical", # clasificación multiclase
    batch_size=32,  # tamaño de batch
    subset="validation",  # Usar esta parte para validación
    fill_mode='nearest'
)


# Calcular pesos de clase para el balanceo. (20 fotos de un ingrediente y 4 del otro)
class_labels = df['label'].astype(str).values
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weight_dict = dict(enumerate(class_weights))

Found 2077 validated image filenames belonging to 109 classes.
Found 519 validated image filenames belonging to 109 classes.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# esto es para prevenir el sobreajuste yo congelo todas las capas de mobilenet y solo extraigo lo que me sirve de mobile net
# que es el reconocimiento de imagenes, no ir cambiando los pesos durante el entrenamiento.
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model, # aca estaria todo lo que es filtrado y agrupamiento (Conv2d y MaxPooling2d)
    # luego del modelo base, hago yo nuevas capas de entrenamiento para mi problema particular.
    # procesamiento de las imagenes y clasificacion.
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(df['label'])), activation='softmax') # agarra los labels unicos es decir una neurona por cada clase
])

# compilacion
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# un resumen si se quiere
# model.summary()

# Early stopping si fuera necesario
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# ir agarrando el mejor modelo a medida que pase el entrenamiento
checkpoint = ModelCheckpoint(
    '/content/drive/My Drive/Proyecto-DataSets/mejorModelo.keras',
    monitor='val_loss',
    save_best_only=True
)



In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    class_weight=class_weight_dict,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 64s 826ms/step - accuracy: 0.1639 - loss: 5.5226 - val_accuracy: 0.0488 - val_loss: 5.2844
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4375 - loss: 8.0492 - val_accuracy: 0.0000e+00 - val_loss: 6.2749
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 67s 664ms/step - accuracy: 0.5512 - loss: 2.7994 - val_accuracy: 0.0430 - val_loss: 6.2225
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4062 - loss: 1.9683 - val_accuracy: 0.0000e+00 - val_loss: 6.3741
Epoch 5/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 46s 663ms/step - accuracy: 0.6696 - loss: 1.9572 - val_accuracy: 0.0430 - val_loss: 7.7455
Epoch 6/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - accuracy: 0.6250 - loss: 1.2909 - val_accuracy: 0.0000e+00 - val_loss: 8.6344
Epoch 7/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 48s 673ms/step - accuracy: 0.7811 - loss: 1.3260 - val_accuracy: 0.0410 - val_loss: 8.2441
Epoch 8/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8750 - loss: 0.7175 - val_a

In [10]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Pérdida en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 574ms/step - accuracy: 0.0433 - loss: 5.3131
Pérdida en validación: 5.298048496246338
Precisión en validación: 0.04816955700516701


In [11]:
model_save_path = '/content/drive/My Drive/Proyecto-DataSets/modeloEntrenado.keras'
model.save(model_save_path)